# Лабораторная работа №7. Классификация

Выполните следующие задания:

1. Откройте в файл в Google Colab (используйте собственный форк репозитория).
2. Решите задачи.
3. Сохраните результат в виде файла rep.ipynb в ваш репозиторий github в директорию ./les07
4. Создайте pull request в репозиторий https://github.com/chebotarevsa/dap-2024. Название pull request должно иметь формат "<Номер лабораторной работы>  <Номер группы> <ФИО>"
5. Сдайте работу в системе "Пегас", в отчет укажите ссылку на pull request

Используя набор данных titanic.csv постройте модель предсказывающую выжил или погиб пассажир.
1.	PassengerId – Идентификатор пассажира (уникальный номер для каждого пассажира).
2.	Survived – Выжил (0 – не выжил, 1 – выжил).
3.	Pclass – Класс пассажира (1 – первый класс, 2 – второй класс, 3 – третий класс).
4.	Name – Имя (полное имя пассажира).
5.	Sex – Пол (male – мужчина, female – женщина).
6.	Age – Возраст (числовое значение, может быть дробным).
7.	SibSp – Количество родственников на борту (братьев, сестер или супругов).
8.	Parch – Количество родителей или детей на борту.
9.	Ticket – Номер билета.
10.	Fare – Стоимость билета (в фунтах стерлингов).
11.	Cabin – Номер каюты (может быть пропущен, если данные отсутствуют).
12.	Embarked – Порт посадки (C – Cherbourg, Q – Queenstown, S – Southampton).

In [ ]:
import sys
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

print(f"Python version: {sys.version}")
print(f"Numpy version: {np.version.version}")
print(f"Pandas version: {pd.__version__}")
print(f"Matplotlib version: {mpl.__version__}")

## Извлечение данных

1. Создайте DataFarame из файла titanic.csv, в качестве индекса используйте столбец "PassengerId".

In [ ]:
import pandas as pd

df = pd.read_csv('titanic.csv', index_col='PassengerId')
print(df.head())



2. Выведете первые 7 строк набора.

In [ ]:
print(df.head(7))


### Подготовка данных

3. Выведете информацию о типах данных в наборе. Имеются ли в наборе не числовые признаки? Имеются ли в наборе данные имеющие значение null?
    

In [ ]:
print(df.dtypes)
print("\nNon-numeric features:", df.select_dtypes(include=['object']).columns.tolist())
print("\nNull values per column:\n", df.isnull().sum())

4. Удалите не числовые признаки, которые по вашему мнению, не могут влиять на заначение выжил или погиб пассажир.

In [ ]:
df = df.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)
print("Updated columns:", df.columns.tolist())

5. Вместо признака "Sex" (я надеюсь вы его не удалили 😂) Создайте два новых признака male и female которые содержат значения 0 или 1.

In [ ]:
df['male'] = (df['Sex'] == 'male').astype(int)
df['female'] = (df['Sex'] == 'female').astype(int)
df = df.drop('Sex', axis=1)
print("Updated columns:", df.columns.tolist())
print(df[['male', 'female']].head())

6. Удалите строки, которые содержать хотя бы одно null значение.

In [ ]:
df = df.dropna()
print(f"Shape after dropna: {df.shape}")

## Исследование данных

4. Нормализуйте значения признака "Fare".

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df['Fare'] = scaler.fit_transform(df[['Fare']])
print(df['Fare'].describe())

5. Найдите разницу между средними значениями признака "Fare" для погибших и выживших пассажиров.

In [ ]:
mean_fare_survived = df[df['Survived'] == 1]['Fare'].mean()
mean_fare_died = df[df['Survived'] == 0]['Fare'].mean()
difference = mean_fare_survived - mean_fare_died
print(f"Mean Fare (Survived): {mean_fare_survived:.3f}")
print(f"Mean Fare (Died): {mean_fare_died:.3f}")
print(f"Difference: {difference:.3f}")

6. Простройте на одной оси координат гистограмы значений признака "Fare" для погибших и выживших пассажиров.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(df[df['Survived'] == 0]['Fare'], bins=30, alpha=0.7, label='Died (Survived=0)', color='red')
plt.hist(df[df['Survived'] == 1]['Fare'], bins=30, alpha=0.7, label='Survived (Survived=1)', color='green')
plt.xlabel('Fare (Normalized)')
plt.ylabel('Frequency')
plt.title('Histograms of Fare for Died and Survived Passengers')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

7. Сформируйте набор признаков (X). Сформируйте вектор целевых значений (y).

In [ ]:
X = df.drop('Survived', axis=1)
y = df['Survived']
print("X shape:", X.shape)
print("y shape:", y.shape)
print("X columns:", X.columns.tolist())

## Предсказательная модель

8. Разделите набор данных на два, одни для обучения модели другой для проверки. Тестовый набор должен содержать 25 процентов данных.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

9. Выполните обучение модели.

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)
print("Model trained")

## Проверка модели

10. Выведите мартицу ошибок

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

11. Расчитайте accuracy

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.3f}")

12. Сделайте вывод о полученном результате

Вывод: Модель логистической регрессии достигла точности 0.771 на тестовом наборе, что превышает baseline (0.62 для угадывания majority-класса погибших) и указывает на хорошую общую предсказательную способность. Матрица ошибок показывает 91 верно предсказанных погибших и 47 выживших, с 16 ложными положительными (ошибочно спасённые) и 25 ложными отрицательными (упущенные выжившие), что отражает небольшую предвзятость к классу "погибшие" из-за дисбаланса (около 60% в тесте). Разница в средних значениях нормализованного Fare (0.101 для выживших против 0.045 для погибших) подтверждает влияние стоимости билета на выживание, как видно на гистограмме (выжившие смещены к высоким значениям)

## Вопросы для защиты

1. Сформулируйте задачу классификации.
2. Перечислите типы классификации.
3. В чем особенность несбалансированной классификации?
4. В чем особенность мультиклассовой классификации?
5. В чем особенность бинарной классификации?
6. В чем особенность классификации по нескольким меткам?
7. Чем стратегия "Один против всех" отличается от стратегии "Один против одного"?
8. Что такое матрица ошибок (несоответствий)?
9. Как рассчитывается Accuracy?
10. Объясните алгоритм классификации K-ближайших соседей.